###Bài 3

In [1]:
from google.colab import drive
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
import numpy as np

drive.mount('/content/drive')

drive_zip_path = '/content/drive/My Drive/VinaFood21.zip'

!mkdir -p ./data
!unzip -q -o "{drive_zip_path}" -d ./data

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

DATA_DIR = './data/VinaFood21'
TRAIN_DIR = os.path.join(DATA_DIR, 'train')
TEST_DIR = os.path.join(DATA_DIR, 'test')

Mounted at /content/drive


In [2]:
input_size = 224

data_transforms = {
    'train': transforms.Compose([
        transforms.Lambda(lambda img: img.convert("RGB")),
        transforms.RandomResizedCrop(input_size),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Lambda(lambda img: img.convert("RGB")),
        transforms.Resize(input_size + 32), # 256
        transforms.CenterCrop(input_size),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

image_datasets = {
    'train': datasets.ImageFolder(TRAIN_DIR, data_transforms['train']),
    'test': datasets.ImageFolder(TEST_DIR, data_transforms['test'])
}

dataloaders = {
    'train': DataLoader(image_datasets['train'], batch_size=128, shuffle=True, num_workers=2),
    'test': DataLoader(image_datasets['test'], batch_size=128, shuffle=False, num_workers=2)
}

dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'test']}
class_names = image_datasets['train'].classes
NUM_CLASSES = len(class_names)

print(f"Số lớp (NUM_CLASSES): {NUM_CLASSES}")
print(f"Kích thước tập huấn luyện: {dataset_sizes['train']}")
print(f"Kích thước tập test: {dataset_sizes['test']}")

Số lớp (NUM_CLASSES): 21
Kích thước tập huấn luyện: 10044
Kích thước tập test: 6682


In [3]:
class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_channels, out_channels, stride=1):
        super(BasicBlock, self).__init__()

        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3,
                               stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)

        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != self.expansion * out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, self.expansion * out_channels,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion * out_channels)
            )

    def forward(self, x):
        identity = self.shortcut(x)

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        out += identity
        out = self.relu(out)
        return out

class ResNet18(nn.Module):
    def __init__(self, block, num_blocks, num_classes=21):
        super(ResNet18, self).__init__()
        self.in_channels = 64

        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)

        self.maxpool1 = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=1)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=1)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=1)

        self.maxpool_custom = nn.MaxPool2d(kernel_size=3, stride=2, padding=0)

        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * block.expansion, num_classes)

    def _make_layer(self, block, out_channels, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for s in strides:
            layers.append(block(self.in_channels, out_channels, s))
            self.in_channels = out_channels * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.relu(self.bn1(self.conv1(x)))
        out = self.maxpool1(out)

        out = self.layer1(out)
        out = self.maxpool_custom(out)

        out = self.layer2(out)
        out = self.maxpool_custom(out)

        out = self.layer3(out)
        out = self.maxpool_custom(out)

        out = self.layer4(out)

        out = self.avgpool(out)
        out = torch.flatten(out, 1)
        out = self.fc(out)

        return out

In [5]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="PIL")

model = ResNet18(BasicBlock, [2, 2, 2, 2], num_classes=NUM_CLASSES).to(DEVICE)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)

for epoch in range(25):

    model.train()
    running_loss = 0.0

    for inputs, labels in dataloaders['train']:
        inputs = inputs.to(DEVICE)
        labels = labels.to(DEVICE)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)

    epoch_loss = running_loss / dataset_sizes['train']
    print(f'Epoch [{epoch+1}/{25}], Train Loss: {epoch_loss:.4f}')

    model.eval()
    all_labels = []
    all_preds = []

    with torch.no_grad():
        for inputs, labels in dataloaders['test']:
            inputs = inputs.to(DEVICE)
            labels = labels.to(DEVICE)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            all_labels.extend(labels.cpu().numpy())
            all_preds.extend(preds.cpu().numpy())

    scheduler.step()
    print(f"Learning Rate hiện tại: {scheduler.get_last_lr()[0]:.10f}")

accuracy = accuracy_score(all_labels, all_preds)
precision = precision_score(all_labels, all_preds, average='weighted', zero_division=0)
recall = recall_score(all_labels, all_preds, average='weighted', zero_division=0)
f1 = f1_score(all_labels, all_preds, average='weighted', zero_division=0)

print(f"Độ chính xác (Accuracy): {accuracy * 100:.2f}%")
print(f"Precision (Macro): {precision:.4f}")
print(f"Recall (Macro): {recall:.4f}")
print(f"F1-score (Macro): {f1:.4f}")

Epoch [1/25], Train Loss: 2.6480
Learning Rate hiện tại: 0.0010000000
Epoch [2/25], Train Loss: 2.2795
Learning Rate hiện tại: 0.0010000000
Epoch [3/25], Train Loss: 2.1325
Learning Rate hiện tại: 0.0010000000
Epoch [4/25], Train Loss: 2.0001
Learning Rate hiện tại: 0.0010000000
Epoch [5/25], Train Loss: 1.9013
Learning Rate hiện tại: 0.0005000000
Epoch [6/25], Train Loss: 1.7178
Learning Rate hiện tại: 0.0005000000
Epoch [7/25], Train Loss: 1.6351
Learning Rate hiện tại: 0.0005000000
Epoch [8/25], Train Loss: 1.5827
Learning Rate hiện tại: 0.0005000000
Epoch [9/25], Train Loss: 1.5281
Learning Rate hiện tại: 0.0005000000
Epoch [10/25], Train Loss: 1.4444
Learning Rate hiện tại: 0.0002500000
Epoch [11/25], Train Loss: 1.3307
Learning Rate hiện tại: 0.0002500000
Epoch [12/25], Train Loss: 1.2688
Learning Rate hiện tại: 0.0002500000
Epoch [13/25], Train Loss: 1.2425
Learning Rate hiện tại: 0.0002500000
Epoch [14/25], Train Loss: 1.1965
Learning Rate hiện tại: 0.0002500000
Epoch [15/25], 